# Log Analysis and Visulization

In [ ]:
import utils
import os
from os import path
import datetime
import pandas as pd

In [ ]:
if __name__ == "__main__":
    
    date = datetime.date.today()-datetime.timedelta(days=1)
    name = "mainnet_"+date.strftime("%Y_%m_%d")
    fig_dir = "../../graphs/test_logs/mainnet/" + date.strftime("%Y_%m_%d") + "/"
    log_dir = "/home/ubuntu/jupyter/monitor/mini_explorer/data/" + name
    html_dir = "../../docs/graphs/test_logs/mainnet/" + date.strftime("%Y_%m_%d") + "/"

    data = [html_dir, fig_dir]
    for directory in data:
        if not path.exists(directory):
            try:
                os.makedirs(directory)
            except:
                print("Could not make directory")
                exit(1)

    data = utils.read_data(log_dir)
    shard = utils.data_processing(data)
    
    utils.set_config({
        'ignore_printing_statistics_summary': True,
        'ignore_drawing_features_vs_time_per_shard' : True,
        'ignore_drawing_features_vs_block_per_shard' : True,
        'ignore_drawing_staking_features_vs_time_per_shard' : True,
        'ignore_drawing_staking_features_vs_block_per_shard' : True,
        'ignore_drawing_features_vs_time' : False,
        'ignore_drawing_features_vs_block' : True
    })
    
    config = utils.config
    ## get the index of the new chain started
    index = []
    for s in shard:
        i = s[s['time_per_block'] < 0].index.tolist()   
        if len(i) == 0:
            break
        i.insert(0,0)
        i.append(len(s))
        index.append(i)
    
    if len(index) == 0:
        new = pd.concat(shard)
        new.reset_index(inplace = True, drop = True)
        print("transaction for ", date, new['transactions'].sum())
        utils.visualization(new, fig_dir, html_dir, config)
    else:    
        for i in range(len(index[0])-1):
            df = []
            for s in range(len(shard)):
                df.append(shard[s].iloc[index[s][i]+1:index[s][i+1]])
            new = pd.concat(df)

            fig_path = fig_dir + "chain_" + str(i) + "/"
            if not os.path.exists(fig_path):
                os.makedirs(fig_path)

            utils.visualization(new, fig_path, html_dir, config)